In [14]:
# !pip install openpyxl

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

In [2]:
df = pd.read_csv("/data/Augmented_data.csv")
df.shape

(234501, 18)

In [3]:
df[df["Exited"] == 1].shape[0] /df.shape[0]

0.14290770615050682

In [4]:
df.head()

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Complain,Satisfaction Score,Card Type,Point Earned,date
0,15572489,HEALEY,1720.007287,Spain,Female,73,29,393987.727351,1,1,1,1.0,0,0,3,DIAMOND,2531.0,1-6-2022
1,15800464,PARTIDA,1518.782461,Spain,Male,53,15,274502.817582,2,1,0,1.0,1,0,4,SILVER,1725.0,1-2-2022
2,15580134,Crawford,502.950000,Spain,Male,27,2,184535.891500,1,1,1,1.0,0,0,3,DIAMOND,711.0,1-1-2020
3,15074588,PINKERTON,1468.954354,Spain,Male,49,15,0.000000,1,1,0,1.0,0,0,5,SILVER,1512.0,1-1-2022
4,15982065,ABERNATHY,1683.914742,France,Male,49,30,0.000000,2,1,0,1.0,1,0,3,DIAMOND,1296.0,1-6-2022


In [5]:
cities = pd.read_excel("US_cities.xlsx")
# companies = pd.read_csv("us_companies.csv")

In [9]:
state_city = {}
for st, ct in zip(cities["state"], cities["city"]):
    if st not in state_city:
        state_city[st] = [ct]
    else:
        state_city[st].append(ct)

In [17]:
df["Surname"] = df["Surname"].progress_apply(lambda x: str(x).capitalize())

100%|██████████| 234501/234501 [00:00<00:00, 1015473.65it/s]


In [22]:
state_mapper = {}
def get_state(x):
    if x not in state_mapper:
        state = np.random.choice(list(state_city.keys()))
        state_mapper[x] = state
        return state
    else:
        return state_mapper[x]
    
df["Geography"] = df["Geography"].progress_apply(get_state)

100%|██████████| 234501/234501 [00:00<00:00, 1213820.53it/s]


In [23]:
df

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Complain,Satisfaction Score,Card Type,Point Earned,date
0,15572489,Healey,1720.007287,West Virginia,Female,73,29,393987.727351,1,1,1,1.0,0,0,3,DIAMOND,2531.0,1-6-2022
1,15800464,Partida,1518.782461,West Virginia,Male,53,15,274502.817582,2,1,0,1.0,1,0,4,SILVER,1725.0,1-2-2022
2,15580134,Crawford,502.950000,West Virginia,Male,27,2,184535.891500,1,1,1,1.0,0,0,3,DIAMOND,711.0,1-1-2020
3,15074588,Pinkerton,1468.954354,West Virginia,Male,49,15,0.000000,1,1,0,1.0,0,0,5,SILVER,1512.0,1-1-2022
4,15982065,Abernathy,1683.914742,Virginia,Male,49,30,0.000000,2,1,0,1.0,1,0,3,DIAMOND,1296.0,1-6-2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234496,15244801,King,1637.900927,Virginia,Male,65,37,0.000000,1,1,1,1.0,0,0,5,GOLD,643.0,1-10-2022
234497,15047568,Medley,612.439174,Virginia,Male,32,1,172743.096102,1,0,0,0.0,0,0,1,DIAMOND,722.0,1-4-2020
234498,15617037,Wyman,855.634870,West Virginia,Male,37,2,213987.894170,1,1,0,1.0,0,0,5,SILVER,870.0,1-10-2020
234499,15788634,Romani,826.575000,West Virginia,Female,37,2,124219.939284,1,0,0,0.0,0,0,5,DIAMOND,1018.0,1-2-2020
